In [1]:
import paramiko
import sys
import textfsm
from tabulate import tabulate
import datetime as time

In [2]:
jump_server_address = '10.251.6.31'   # The internal ip Address for the Jump server
local_IP_address = '127.0.0.1'  # ip Address of the machine you are connecting from
username = ""
password = ""
debug = 1

In [3]:
def error_log(message,):
    date_time_object = time.datetime.now()
    datetime = date_time_object.strftime("%d/%m/%Y %H:%M:%S")
    error_file = open("Error Log.txt", "a")
    error_file.write(f"{datetime} - {message}")
    error_file.write("\n")
    error_file.close()
    if debug == 1:
        print(message)


def output_log(message,):
    date_time_object = time.datetime.now()
    datetime = date_time_object.strftime("%d/%m/%Y %H:%M:%S")
    output_file = open("Output Log.txt", "a")
    output_file.write(f"{datetime} - {message}")
    output_file.write("\n")
    output_file.close()
    if debug == 1:
        print(message)


def open_session(ip):
    try:
        output_log(f"Trying to establish a connection to: {ip}")
        jump_box = paramiko.SSHClient()
        jump_box.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        jump_box.connect(jump_server_address, username=username, password=password)
        jump_box_transport = jump_box.get_transport()
        src_address = (local_IP_address, 22)
        destination_address = (ip, 22)
        jump_box_channel = jump_box_transport.open_channel("direct-tcpip", destination_address, src_address)
        target = paramiko.SSHClient()
        target.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        target.connect(destination_address, username=username, password=password, sock=jump_box_channel)
        output_log(f"Connection to ip: {ip} established")
        return target, jump_box, True
    except paramiko.ssh_exception.AuthenticationException:
        error_log(f"Authentication to ip: {ip} failed! Please check your ip, username and password.")
        return None, None, False
    except paramiko.ssh_exception.NoValidConnectionsError:
        error_log(f"Unable to connect to ip: {ip}!")
        return None, None, False
    except (ConnectionError, TimeoutError):
        error_log(f"Timeout error occurred for ip: {ip}!")
        return None, None, False
    except Exception as err:
        error_log(f"Open Session Error: An unknown error occurred for ip: {ip}!")
        error_log(f"\t Error: {err}")
        return None, None, False

In [4]:
command = "sh cdp neighbors detail"
ssh, jumpbox, connection = open_session("10.145.61.10")
stdin, stdout, stderr = ssh.exec_command(command)
stdout = stdout.read()
stdout = stdout.decode("utf-8")
ssh.close()
jumpbox.close()

Trying to establish a connection to: 10.145.61.10
Connection to ip: 10.145.61.10 established


In [5]:
template = "cdp_detail_config.txt"

In [14]:
with open(template) as f:
    re_table = textfsm.TextFSM(f)
    header = re_table.header
    result = re_table.ParseText(stdout)
    print(tabulate(result, headers=header))

LOCAL_HOST    DEST_HOST                               MGMNT_IP     PLATFORM           LOCAL_PORT             REMOTE_PORT               IOS_VERSION
------------  --------------------------------------  -----------  -----------------  ---------------------  ------------------------  -------------
              GB-CAY2-001CSW001.cns.muellergroup.com  10.145.61.1  cisco C9300-48UXM  GigabitEthernet1/0/49  TenGigabitEthernet1/0/48  16.12.4


In [13]:
re_table.ParseText(stdout)

[['',
  'GB-CAY2-001CSW001.cns.muellergroup.com',
  '10.145.61.1',
  'cisco C9300-48UXM',
  'GigabitEthernet1/0/49',
  'TenGigabitEthernet1/0/48',
  '16.12.4'],
 ['',
  'GB-CAY2-001CSW001.cns.muellergroup.com',
  '10.145.61.1',
  'cisco C9300-48UXM',
  'GigabitEthernet1/0/49',
  'TenGigabitEthernet1/0/48',
  '16.12.4'],
 ['',
  'GB-CAY2-001CSW001.cns.muellergroup.com',
  '10.145.61.1',
  'cisco C9300-48UXM',
  'GigabitEthernet1/0/49',
  'TenGigabitEthernet1/0/48',
  '16.12.4'],
 ['',
  'GB-CAY2-001CSW001.cns.muellergroup.com',
  '10.145.61.1',
  'cisco C9300-48UXM',
  'GigabitEthernet1/0/49',
  'TenGigabitEthernet1/0/48',
  '16.12.4']]